<a href="https://colab.research.google.com/github/rajibmondal/Projects.NLP/blob/master/01_transformers_multiclass_classification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [0]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
# !pip install transformers
from transformers import DistilBertModel, DistilBertTokenizer

In [0]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

##Download Dara from Another repositary and remove onters unnecessary data

In [4]:
#download data from another repositary
! git clone https://github.com/PacktPublishing/Apache-Spark-2x-Machine-Learning-Cookbook.git

fatal: destination path 'Apache-Spark-2x-Machine-Learning-Cookbook' already exists and is not an empty directory.


In [5]:
cd/content/Apache-Spark-2x-Machine-Learning-Cookbook/

/content/Apache-Spark-2x-Machine-Learning-Cookbook


In [0]:
!rm -rf Chapter01 Chapter02 Chapter07 Chapter10 Chapter13 Chapter05 Chapter08 Chapter11 LICENSE Chapter03 Chapter06 Chapter09 Chapter12 README.md

In [7]:
cd/content/Apache-Spark-2x-Machine-Learning-Cookbook/Chapter04/

/content/Apache-Spark-2x-Machine-Learning-Cookbook/Chapter04


In [0]:
!rm -rf Chapter 4 Chapter\ 4\ Code

In [9]:
cd/content/Apache-Spark-2x-Machine-Learning-Cookbook/Chapter04/Chapter 4 Data

/content/Apache-Spark-2x-Machine-Learning-Cookbook/Chapter04/Chapter 4 Data


In [0]:
!rm -rf beatles.txt my_kmeans_data_sample.txt breast-cancer-wisconsin.data myMulticlassIrisData.txt teams.csv myBinaryClassificationData.txt mySampleCSV.csv wine.data

In [11]:
!ls

newsCorpora.csv


##Load Data

In [0]:
df = pd.read_csv('/content/Apache-Spark-2x-Machine-Learning-Cookbook/Chapter04/Chapter 4 Data/newsCorpora.csv', sep='\t', names=['ID','TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'] )

# # Removing unwanted columns and only leaving title of news and the category which will be the target
df = df[['TITLE','CATEGORY']]

# # Converting the codes to appropriate categories using a dictionary
my_dict = {
    'e':'Entertainment',
    'b':'Business',
    't':'Science',
    'm':'Health'
}

def update_cat(x):
    return my_dict[x]

df['CATEGORY'] = df['CATEGORY'].apply(lambda x: update_cat(x))

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return len(encode_dict)

df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: encode_cat(x))

In [13]:
df.head()

,TITLE,CATEGORY,ENCODE_CAT
0,"Fed official says weak data caused by weather,...",Business,1
1,Fed's Charles Plosser sees high bar for change...,Business,1
2,US open: Stocks fall after Fed official hints ...,Business,1
3,"Fed risks falling 'behind the curve', Charles ...",Business,1
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,Business,1


In [14]:
# df = df[1:100000]
df

,TITLE,CATEGORY,ENCODE_CAT
0,"Fed official says weak data caused by weather,...",Business,1
1,Fed's Charles Plosser sees high bar for change...,Business,1
2,US open: Stocks fall after Fed official hints ...,Business,1
3,"Fed risks falling 'behind the curve', Charles ...",Business,1
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,Business,1
...,...,...,...
422414,Surgeons to remove 4-year-old's rib to rebuild...,Health,4
422415,Boy to have surgery on esophagus after battery...,Health,4
422416,Child who swallowed battery to have reconstruc...,Health,4
422417,Phoenix boy undergoes surgery to repair throat...,Health,4


In [0]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 130#512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [0]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.TITLE[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [17]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (422419, 3)
TRAIN Dataset: (337935, 3)
TEST Dataset: (84484, 3)


In [0]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [0]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 1)
    
    def forward(self, ids, mask):
        output_1= self.l1(ids, mask)
        output_2 = self.l2(output_1[0])
        output = self.l3(output_2)
        return output

In [20]:
model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [0]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [0]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask).squeeze()

        optimizer.zero_grad()
        loss = loss_function(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [23]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  4.938161849975586
Epoch: 0, Loss:  0.0018693208694458008
Epoch: 0, Loss:  0.003926277160644531
Epoch: 0, Loss:  0.005204439163208008
Epoch: 0, Loss:  0.007777214050292969
Epoch: 0, Loss:  0.00227510929107666
Epoch: 0, Loss:  0.006875157356262207
Epoch: 0, Loss:  0.004320740699768066
Epoch: 0, Loss:  0.0004976987838745117
Epoch: 0, Loss:  0.0010658502578735352
Epoch: 0, Loss:  0.0012394189834594727
Epoch: 0, Loss:  0.004031538963317871
Epoch: 0, Loss:  0.0005962848663330078
Epoch: 0, Loss:  0.0011070966720581055
Epoch: 0, Loss:  0.0024118423461914062
Epoch: 0, Loss:  0.00248563289642334
Epoch: 0, Loss:  0.013435602188110352


In [0]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            total+=targets.size(0)
            n_correct+=(big_idx==targets).sum().item()
    return (n_correct*100.0)/total

In [26]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

This is the validation section to print the accuracy and see how it performs
Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch
Accuracy on test data = 99.97%
